In [1]:
# Import the necessary Packages for this software to run
import mediapipe as mp
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
#Create Table
df_columns = ['video_id', 'label_id', 'label']
for frame in range(1,38):
    for landmark in range (0,21):
        s1 = f'{"F"}{frame}{"_L"}{landmark}{"_X"}'
        s2 = f'{"F"}{frame}{"_L"}{landmark}{"_Y"}'
        s3 = f'{"F"}{frame}{"_L"}{landmark}{"_Z"}'
        df_columns.append(s1)
        df_columns.append(s2)
        df_columns.append(s3)


In [3]:
#Create Dataframe
df = pd.DataFrame(columns=df_columns)

In [4]:
#Show Dataframe
df.head()

,video_id,label_id,label,F1_L0_X,F1_L0_Y,F1_L0_Z,F1_L1_X,F1_L1_Y,F1_L1_Z,F1_L2_X,...,F37_L17_Z,F37_L18_X,F37_L18_Y,F37_L18_Z,F37_L19_X,F37_L19_Y,F37_L19_Z,F37_L20_X,F37_L20_Y,F37_L20_Z


In [6]:
# Path to Excel File with Labels
file_path_train = 'C:/Users/zoech/Documents/Master/ML Project/Projektcode/Train_sortiert.xlsx'
file_path_val = 'C:/Users/zoech/Documents/Master/ML Project/Projektcode/Validation_sortiert.xlsx'

# Path to the directory containing the images
root_dir = "C:/Users/zoech/Documents/Master/ML Project/Projektcode/train_klein"

In [7]:
# load excel files
df_train = pd.read_excel(file_path_train)
df_val = pd.read_excel(file_path_val)

In [8]:
# Concatenate Dataframes
frames = [df_train, df_val]
df_labels = pd.concat(frames)

#show new Dataframe
df_labels.head()

,video_id,label,frames,label_id,shape,format
0,14,Pushing Hand Away,37,5,"(100, 176)",JPEG
1,79,Pushing Hand Away,37,5,"(100, 132)",JPEG
2,357,Pushing Hand Away,37,5,"(100, 176)",JPEG
3,361,Pushing Hand Away,37,5,"(100, 176)",JPEG
4,523,Pushing Hand Away,37,5,"(100, 176)",JPEG


In [9]:
# Use MediaPipe to draw the hand framework over the top of hands it identifies
drawingModule = mp.solutions.drawing_utils
handsModule = mp.solutions.hands

# Use MediaPipe hand tracking with static_image_mode set to True
with handsModule.Hands(static_image_mode=True, min_detection_confidence=0.7, max_num_hands=1) as hands:
    for gesture in os.listdir(root_dir):
        gesture_path = os.path.join(root_dir, gesture)
        new_row = [gesture, 0, 0]
    # Loop through all files in the directory
        j = 0
        for filename in os.listdir(gesture_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):  # Add other image formats if necessary
                image_path = os.path.join(gesture_path, filename)
                
                # Read the image
                image = cv2.imread(image_path)
                
                # Determine the frame size, resize if necessary
                frame = cv2.resize(image, (640, 480))
                
                # Process the image and produce the hand framework overlay on top of the hand
                results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

                # In case the system sees multiple hands, this if statement deals with that and produces another hand overlay
                if results.multi_hand_landmarks:
                    #print(len(results.multi_hand_landmarks))
                    
                    
                    for handLandmarks in results.multi_hand_landmarks:
                        drawingModule.draw_landmarks(frame, handLandmarks, handsModule.HAND_CONNECTIONS)
                        
                        # Print the x, y, z coordinates of each landmark
                        i = 0
                        last_idx = 0
                        for idx, landmark in enumerate(handLandmarks.landmark):
                            #print(f'Landmark {idx}: (x={landmark.x}, y={landmark.y}, z={landmark.z})')
                            while (i < idx):
                                new_row.append(0)
                                new_row.append(0)
                                new_row.append(0)
                                i = i+1

                            new_row.append(landmark.x)
                            new_row.append(landmark.y)
                            new_row.append(landmark.z)
                            i = idx+1
                            last_idx = idx
                            #print(idx)
                        if last_idx <= 20:
                            #print("here")
                            for i in range(last_idx+1, 21):
                                new_row.append(0)
                                new_row.append(0)
                                new_row.append(0)
                
                else:
                    #arr = np.zeros(63).astype(np.str_)
                    for i in range (0,63):
                        new_row.append(0)
                    #new_row = new_row + arr
                    j = j+1
                    

                # Convert the image from BGR to RGB (OpenCV uses BGR by default)
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                
                

                # Uncomment the next line if you want to pause between images
                # input("Press Enter to continue to the next image...")
        #while len(new_row) < 2334:
         #  new_row.append(0)

        #extract label values from df_labels
        label_value = df_labels.loc[df_labels['video_id'] == gesture, 'label'].iloc[0]
        label_id_value = df_labels.loc[df_labels['video_id'] == gesture, 'label_id'].iloc[0]
        new_row[1]=label_id_value
        new_row[2]=label_value
        
        #add new line to df
        df.loc[len(df)] = new_row



IndexError: single positional indexer is out-of-bounds

In [10]:
#show dataframe
df.head()

,video_id,label_id,label,F1_L0_X,F1_L0_Y,F1_L0_Z,F1_L1_X,F1_L1_Y,F1_L1_Z,F1_L2_X,...,F37_L17_Z,F37_L18_X,F37_L18_Y,F37_L18_Z,F37_L19_X,F37_L19_Y,F37_L19_Z,F37_L20_X,F37_L20_Y,F37_L20_Z
